# Aprendizado Profundo (Ext. Artibutos RP - DTLFE)

Avaliar pipeline DTLFE utilizando diferentes arquiteturas pré-treinadas.


# Configurações

In [1]:
import os
import sys
import gc
from pprint import pprint
from collections import Counter
import json
import copy
import warnings
warnings.filterwarnings(action="ignore")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display

from tqdm import *

from pretty_confusion_matrix import *

# TODO: implementar rotina na classe PyNILM.utils
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def listar_variaveis_memoria(ambiente):
    print("* Variáveis instanciadas em memória:")
    print("---")
    total = 0
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in ambiente.items()),
                             key= lambda x: -x[1])[:10]:
        total += size
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
    print("---")
    print("Total:", sizeof_fmt(total))
    
# TODO: implementar na classe utils
def highlight_col(x):
    r = 'background-color: #D9D9D9'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, -2] = r
    return df1   

In [2]:
# CONSTANTES FUNDAMENTAIS DE ORGANIZACAO DE PASTAS/ARQUIVOS
RESIDENCIA = 3

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK) e outros insumos fundamentais
caminho_dados = "D:/Projetos/phd-thesis/datasets/"

# Definir diretorios onde iremos salvar os insumos gerados do notebook (dados, imagens, etc.)
caminho_dados_notebook = os.path.join(caminho_dados, "30") # Num. notebook
if not os.path.isdir(caminho_dados_notebook):
    os.makedirs(caminho_dados_notebook)
caminho_imagens_notebook = os.path.join(caminho_dados_notebook, "imagens") # Num. notebook
if not os.path.isdir(caminho_imagens_notebook):
    os.makedirs(caminho_imagens_notebook)

# Path do arquivo H5 (base REDD ja preparada p/ NILMTK)
caminho_redd = os.path.join(caminho_dados, "REDD/low_freq")

# Path completo do arquivo REDD
arquivo_dataset = os.path.join(caminho_redd, "redd.h5")

# VARIAVEL AUXILIAR
# Path dos arquivos relacionados as janelas
caminho_janelas = os.path.join(caminho_redd, "../../phd")
if not os.path.isdir(caminho_janelas):
    os.makedirs(caminho_janelas)

In [3]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
from six import iteritems

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation, FHMM
import nilmtk.utils

%matplotlib inline

# Dados

## Base REDD

In [4]:
# Gerar arquivo H5 (Nilmtk) do dataset REDD, caso n exista
if not os.path.isfile(arquivo_dataset):
    from nilmtk.dataset_converters import convert_redd
    
    print("Gerando arquivo H5 (NILMTK) da base REDD, aguarde...")
    print("-----")
    convert_redd(caminho_redd, arquivo_dataset)

# Carregando dataset REDD no objeto NILMTK
# Exemplo de carregamento da base REDD no NILMTK
import h5py # * Evitar erro de incompatibilidade entre h5py e nilmtk
from nilmtk import DataSet
from nilmtk.utils import print_dict
redd = DataSet(arquivo_dataset)
print("NILMTK -> Detalhes sobre o dataset REDD:")
print_dict(redd.metadata)
print()

NILMTK -> Detalhes sobre o dataset REDD:


## Melhores Combinações de Taxas e Janelas para cada Aparelho (artigo IEEE-2021)

In [5]:
df_melhores_taxas_janelas = pd.read_csv(
    os.path.join(caminho_dados, "27", "melhores_taxa_janela_aparelhos_ieee2021.csv"), 
    index_col=0)[['carga', 'taxa_amostragem', 'janela']]
df_melhores_taxas_janelas

,carga,taxa_amostragem,janela
0,dish_washer - 9,2,2040
1,fridge - 7,2,720
2,microwave - 16,2,900
3,washer_dryer - 13,2,90
4,washer_dryer - 14,3,2040


In [7]:
# Melhores configurações de janelas por aparelhos, conforme 
#  artigo IEEE (https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/6287639/9312710/9564044/ferna.t3-3118947-large.gif),
#  independente do nomenclatura/instancia da base REDD.
with open(
    os.path.join(caminho_dados, "27", "config_aparelhos_janelas_ieee2021.json"), 
    'r') as f:
    aparelhos_janelas = json.load(f)
aparelhos_janelas
# * obs.: como a base redd (artigo) possui dois washer_dryer's, onde verificou-se
#         duas configurações distintas de janelas (90 e 2040), vamos usar a maior.

{'dish': 2040,
 'fridge': 720,
 'microwave': 900,
 'washer_dryer': 2040,
 'washing_machine': 2040}

In [6]:
# TODO: 
# - Desenvolver módulo da metodologia na lib PyNILM

## Parâmetros de RP dos Aparelhos (estudo 18)

In [8]:
# Carregando arquivos de parametros, caso n estejam (kernel reiniciado)
if not 'parametros_rp_aparelho' in locals():
    with open(os.path.join(caminho_dados, "18", "parametros_rp_aparelho.json"),'r') as arquivo:
        parametros_rp_aparelho = json.load(arquivo)

## Ambiente e Funções Auxiliares

In [9]:
# from PyNILM.dados.janelas import Janelas
from PyNILM.dados.janelas import Janelas
from PyNILM.dados.utils import *

from PyNILM.avaliacao.metricas import *
from PyNILM.avaliacao.graficos import *
from PyNILM.avaliacao.analises import *

from PyNILM.modelos.utils import *
from PyNILM.modelos.dlafe import DLAFE
from PyNILM.modelos.rqa import RQA

# Inicializar uso GPU
start_tf_session(memory_limit=int(1024*4))

Virtual devices cannot be modified after being initialized


## Configurações do Experimento

In [10]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost.sklearn import XGBClassifier

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

# Teste da classe
janelas_treino = Janelas(
    base=DataSet(arquivo_dataset),
    id_residencia=3,
    inicio_intervalo='2011-04-16 00:00:00',
    fim_intervalo='2011-05-16 23:59:59',
    debug = False
)

janelas_teste = Janelas(
    base=DataSet(arquivo_dataset),
    id_residencia=3,
    inicio_intervalo='2011-05-17 00:00:00',
    fim_intervalo='2011-05-30 23:59:59',
    debug = False
)

aparelhos = [
    'dish_washer - 9',
    'fridge - 7',
    'microwave - 16',
    'washer_dryer - 13', 
    'washer_dryer - 14'
]

TAXA = 2 # Fixa

# modelo_extrator = transfer_learning.vgg16.VGG16(
#             weights='imagenet', 
#             include_top=False,
#             pooling='avg'
#         )
# preprocessamento_extrator = transfer_learning.vgg16.preprocess_input

# Metodologia DLAFE

In [11]:
# from keras.applications import convnext
from tensorflow.keras.applications import densenet
from tensorflow.keras.applications import efficientnet
# from tensorflow.keras.applications import inception_resnet_v2
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.applications import mobilenet_v2
# from tensorflow.keras.applications import mobilenet_v3
from tensorflow.keras.applications import nasnet
# from tensorflow.keras.applications import regnet
from tensorflow.keras.applications import resnet
# from tensorflow.keras.applications import resnet_rs
from tensorflow.keras.applications import resnet_v2
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import vgg19
from tensorflow.keras.applications import xception

# Carregando diferentes arquiteturas/modelos disponíveis no tf.keras
# Ref:
# - https://keras.io/api/applications/ 
# - https://github.com/keras-team/keras/blob/7dd01b12c275d8f5f9c1f82014deb302ed14560c/keras/applications/applications_load_weight_test.py#L42
arquiteturas_deeplearning = {
    "resnet": (resnet, [resnet.ResNet50, resnet.ResNet101, resnet.ResNet152]),
    "resnet_v2": (
        resnet_v2,
        [resnet_v2.ResNet50V2, resnet_v2.ResNet101V2, resnet_v2.ResNet152V2],
    ),
    "vgg16": (vgg16, [vgg16.VGG16]),
    "vgg19": (vgg19, [vgg19.VGG19]),
    "xception": (xception, [xception.Xception]),
    "inception_v3": (inception_v3, [inception_v3.InceptionV3]),
    "mobilenet": (mobilenet, [mobilenet.MobileNet]),
    "mobilenet_v2": (mobilenet_v2, [mobilenet_v2.MobileNetV2]),
    "densenet": (
        densenet,
        [densenet.DenseNet121, densenet.DenseNet169, densenet.DenseNet201],
    ),
    "nasnet_mobile": (nasnet, [nasnet.NASNetMobile]),
    "nasnet_large": (nasnet, [nasnet.NASNetLarge]),
    "efficientnet": (
        efficientnet,
        [
            efficientnet.EfficientNetB0,
            efficientnet.EfficientNetB1,
            efficientnet.EfficientNetB2,
            efficientnet.EfficientNetB3,
            efficientnet.EfficientNetB4,
            efficientnet.EfficientNetB5,
            efficientnet.EfficientNetB6,
            efficientnet.EfficientNetB7,
        ],
    ),
}

In [12]:
TAMANHO_IMAGEM_DLAFE = (128, 128, 3)

## SVM

In [13]:
modelo = SVC(kernel='rbf', random_state=SEED)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": [],
    "arquitetura": [] # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
}

for a in arquiteturas_deeplearning.keys():
    # Preprocessamento da arquitetura
    preprocessamento_extrator = arquiteturas_deeplearning[a][0].preprocess_input
    # Iterando multiplos modelos da arquitetura
    for arquitetura in arquiteturas_deeplearning[a][1]:
        print(f'# Avaliando modelo extrator `{arquitetura.__name__}`...\n')
        try:
            modelo_extrator = arquitetura(
                weights='imagenet', 
                include_top=False,
                pooling='avg'
            )

            for rotulo_aparelho in aparelhos:
                
                print(f"* Aparelho `{rotulo_aparelho}`...\n")
                
                # Informacoes da carga selecionada
                CARGA = rotulo_aparelho.split(" - ")[0]
                INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
                config_aparelho = df_melhores_taxas_janelas[
                    df_melhores_taxas_janelas["carga"]==rotulo_aparelho
                ].to_dict("records")[0]
                TAMANHO_JANELA = config_aparelho["janela"]

                #######################################################################
                #                AVALIACAO 1 - Base de treino / CV                    #
                #######################################################################
                # Extrair series divididas em janelas para cada medidor
                print("   - Base de TREINO\n")
                print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
                    TAXA, TAMANHO_JANELA
                ))
                X, y = carregar_dados_aparelho(
                    janelas=janelas_treino,
                    instancia=INSTANCIA,
                    aparelho=CARGA,
                    tamanho_janela=TAMANHO_JANELA,
                    taxa=TAXA,
                    eliminar_janelas_vazias=True
                )
                print("     -> Detalhes da amostragem (lotes):")
                print("     ---")
                for item in Counter(y).items():
                    print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
                print()
                
                y_true, y_pred  = [], []

                print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
                for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X, y)), total=skf.n_splits):

                    # Preparando lotes
                    X_treino, X_teste = X[idx_treino], X[idx_teste]
                    y_treino, y_teste = y[idx_treino], y[idx_teste]

                    # Treinando modelo
                    dlafe = DLAFE(
                        feature_extractor=modelo_extrator,
                        preprocess_input=preprocessamento_extrator,
                        classifier=clone(modelo),
                        rp_params = PARAMETROS_RP,
                        input_shape = TAMANHO_IMAGEM_DLAFE,
                        normalize=False
                    )
                    dlafe.fit(X_treino, y_treino)

                    # Prevendo conjunto de teste
                    y_hat = dlafe.predict(X_teste)

                    # Incrementando resultados
                    resultados_modelo["appliance"].append(rotulo_aparelho)
                    resultados_modelo["fold"].append(it+1)
                    resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
                    resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
                    resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
                    resultados_modelo["base"].append("treino")
                    resultados_modelo["arquitetura"].append(arquitetura.__name__) # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                    
                    reset_tf_session(model_name='dlafe')

                    # Extendendo rotulos (analise global)
                    y_true.extend(y_teste)
                    y_pred.extend(y_hat)
                    
                #######################################################################
                #                 AVALIACAO 2 - Base de teste / CV                    #
                #######################################################################
                print("   - Base de TESTE\n")
                print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
                    TAXA, TAMANHO_JANELA
                ))

                # Avaliar na base de teste
                X_teste, y_teste = carregar_dados_aparelho(
                    janelas=janelas_teste,
                    instancia=INSTANCIA,
                    aparelho=CARGA,
                    tamanho_janela=TAMANHO_JANELA,
                    taxa=TAXA,
                    eliminar_janelas_vazias=True
                )
                
                print("     -> Detalhes da amostragem (lotes):")
                print("     ---")
                for item in Counter(y_teste).items():
                    print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_teste)*100,1)}%)" )
                print()

                # Treinando modelo
                dlafe = DLAFE(
                    feature_extractor=modelo_extrator,
                    preprocess_input=preprocessamento_extrator,
                    classifier=clone(modelo),
                    rp_params = PARAMETROS_RP,
                    input_shape = TAMANHO_IMAGEM_DLAFE,
                    normalize=False
                )
                dlafe.fit(X, y)

                # Prevendo conjunto de teste
                y_hat = dlafe.predict(X_teste)

                # Incrementando resultados
                resultados_modelo["appliance"].append(rotulo_aparelho)
                resultados_modelo["fold"].append(it+1)
                resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
                resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
                resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
                resultados_modelo["base"].append("teste")
                resultados_modelo["arquitetura"].append(arquitetura.__name__) # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                
                reset_tf_session(model_name='dlafe')
                
                print()
                print("   - Final Results:")
                print("   ---")
                print()
                
                print("***** TRAIN *****")
                print("      -> Classification Report:")
                print()
                print(classification_report(y_true, y_pred))
                print("      -> Confusion Matrix:")
                print()
                print(confusion_matrix(y_true, y_pred))
                print()
                
                print("***** TEST *****")
                print("      -> Classification Report:")
                print()
                print(classification_report(y_teste, y_hat))
                print("      -> Confusion Matrix:")
                print()
                print(confusion_matrix(y_teste, y_hat))
                print()

        except Exception as e:
            print(f"Não foi possível carregar o modelo {arquitetura.__name__}: {str(e)}")
    
# Consolidating DataFrame
df_resultados = pd.DataFrame(resultados_modelo)
df_resultados.to_excel(os.path.join(caminho_dados_notebook, "df_resultados_svm.xlsx"))
    
print("############################## FINAL MODEL RESULTS ##############################")
display(df_resultados.groupby(["appliance","base", "arquitetura"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))

# Avaliando modelo extrator `ResNet50`...

* Aparelho `dish_washer - 9`...

   - Base de TREINO

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
        - Classe `0`: 217 amostras (94.8%)
        - Classe `1`: 12 amostras (5.2%)

     -> Avaliando modelo (CV - 10 folds)...



  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 138 amostras (93.2%)
       - Classe `1`: 10 amostras (6.8%)

Virtual devices cannot be modified after being initialized

   - Final 

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=720)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `1`: 392 amostras (95.6%)
       - Classe `0`: 18 amostras (4.4%)

Virtual devices cannot be modified after being initialized

   - Final R

...

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=90)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 3102 amostras (95.9%)
       - Classe `1`: 132 amostras (4.1%)

Virtual devices cannot be modified after being initialized

   - Final 

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 134 amostras (90.5%)
       - Classe `1`: 14 amostras (9.5%)

Virtual devices cannot be modified after being initialized

   - Final 

acc                      \
                                             mean       std       max   
appliance         base   arquitetura                                    
dish_washer - 9   teste  DenseNet121     0.932432       NaN  0.932432   
                         DenseNet169     0.932432       NaN  0.932432   
                         DenseNet201     0.932432       NaN  0.932432   
                         EfficientNetB0  0.932432       NaN  0.932432   
                         EfficientNetB1  0.932432       NaN  0.932432   
...                                           ...       ...       ...   
washer_dryer - 14 treino ResNet50        0.986957  0.021002  1.000000   
                         ResNet50V2      0.982609  0.030400  1.000000   
                         VGG16           0.986957  0.021002  1.000000   
                         VGG19           0.986957  0.029346  1.000000   
                         Xception        0.978261  0.022915  1.000000   

                                                         f1            \
                                              min      mean       std   
appliance         base   arquitetura                                    
dish_washer - 9   teste  DenseNet121     0.932432  0.482517       NaN   
                         DenseNet169     0.932432  0.482517       NaN   
                         DenseNet201     0.932432  0.482517       NaN   
                         EfficientNetB0  0.932432  0.482517       NaN   
                         EfficientNetB1  0.932432  0.482517       NaN   
...                                           ...       ...       ...   
washer_dryer - 14 treino ResNet50        0.956522  0.946512  0.086124   
                         ResNet50V2      0.913043  0.936960  0.104780   
                         VGG16           0.956522  0.913230  0.166507   
                         VGG19           0.913043  0.954790  0.097935   
                         Xception        0.956522  0.844289  0.204227   

                                                                  auc  \
                                              max       min      mean   
appliance         base   arquitetura                                    
dish_washer - 9   teste  DenseNet121     0.482517  0.482517  0.500000   
                         DenseNet169     0.482517  0.482517  0.500000   
                         DenseNet201     0.482517  0.482517  0.500000   
                         EfficientNetB0  0.482517  0.482517  0.500000   
                         EfficientNetB1  0.482517  0.482517  0.500000   
...                                           ...       ...       ...   
washer_dryer - 14 treino ResNet50        1.000000  0.821705  0.925000   
                         ResNet50V2      1.000000  0.726190  0.922619   
                         VGG16           1.000000  0.488889  0.900000   
                         VGG19           1.000000  0.726190  0.947619   
                         Xception        1.000000  0.488889  0.825000   

                                                                 
                                              std  max      min  
appliance         base   arquitetura                             
dish_washer - 9   teste  DenseNet121          NaN  0.5  0.50000  
                         DenseNet169          NaN  0.5  0.50000  
                         DenseNet201          NaN  0.5  0.50000  
                         EfficientNetB0       NaN  0.5  0.50000  
                         EfficientNetB1       NaN  0.5  0.50000  
...                                           ...  ...      ...  
washer_dryer - 14 treino ResNet50        0.120761  1.0  0.75000  
                         ResNet50V2      0.124764  1.0  0.72619  
                         VGG16           0.174801  1.0  0.50000  
                         VGG19           0.110571  1.0  0.72619  
                         Xception        0.205818  1.0  0.50000  

[230 rows x 12 columns]

## XGBOOST

In [14]:
def pos_weight(y):
    try:
        counter = Counter(y)
        return counter[0]/counter[1]
    except:
        return 1

In [15]:
modelo = XGBClassifier(random_state=SEED, n_jobs=4)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": [],
    "arquitetura": [] # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
}

for a in arquiteturas_deeplearning.keys():
    # Preprocessamento da arquitetura
    preprocessamento_extrator = arquiteturas_deeplearning[a][0].preprocess_input
    # Iterando multiplos modelos da arquitetura
    for arquitetura in arquiteturas_deeplearning[a][1]:
        print(f'# Avaliando modelo extrator `{arquitetura.__name__}`...\n')
        try:
            modelo_extrator = arquitetura(
                weights='imagenet', 
                include_top=False,
                pooling='avg'
            )

            for rotulo_aparelho in aparelhos:
                
                print(f"* Aparelho `{rotulo_aparelho}`...\n")
                
                # Informacoes da carga selecionada
                CARGA = rotulo_aparelho.split(" - ")[0]
                INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
                config_aparelho = df_melhores_taxas_janelas[
                    df_melhores_taxas_janelas["carga"]==rotulo_aparelho
                ].to_dict("records")[0]
                TAMANHO_JANELA = config_aparelho["janela"]

                #######################################################################
                #                AVALIACAO 1 - Base de treino / CV                    #
                #######################################################################
                # Extrair series divididas em janelas para cada medidor
                print("   - Base de TREINO\n")
                print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
                    TAXA, TAMANHO_JANELA
                ))
                X, y = carregar_dados_aparelho(
                    janelas=janelas_treino,
                    instancia=INSTANCIA,
                    aparelho=CARGA,
                    tamanho_janela=TAMANHO_JANELA,
                    taxa=TAXA,
                    eliminar_janelas_vazias=True
                )
                print("     -> Detalhes da amostragem (lotes):")
                print("     ---")
                for item in Counter(y).items():
                    print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
                print()
                
                y_true, y_pred  = [], []

                print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
                for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X, y)), total=skf.n_splits):

                    # Preparando lotes
                    X_treino, X_teste = X[idx_treino], X[idx_teste]
                    y_treino, y_teste = y[idx_treino], y[idx_teste]

                    # Treinando modelo
                    dlafe = DLAFE(
                        feature_extractor=modelo_extrator,
                        preprocess_input=preprocessamento_extrator,
                        classifier=clone(modelo),
                        rp_params = PARAMETROS_RP,
                        input_shape = TAMANHO_IMAGEM_DLAFE,
                        normalize=False
                    )
                    dlafe.fit(X_treino, y_treino)

                    # Prevendo conjunto de teste
                    y_hat = dlafe.predict(X_teste)

                    # Incrementando resultados
                    resultados_modelo["appliance"].append(rotulo_aparelho)
                    resultados_modelo["fold"].append(it+1)
                    resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
                    resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
                    resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
                    resultados_modelo["base"].append("treino")
                    resultados_modelo["arquitetura"].append(arquitetura.__name__) # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                    
                    reset_tf_session(model_name='dlafe')

                    # Extendendo rotulos (analise global)
                    y_true.extend(y_teste)
                    y_pred.extend(y_hat)
                    
                #######################################################################
                #                 AVALIACAO 2 - Base de teste / CV                    #
                #######################################################################
                print("   - Base de TESTE\n")
                print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
                    TAXA, TAMANHO_JANELA
                ))

                # Avaliar na base de teste
                X_teste, y_teste = carregar_dados_aparelho(
                    janelas=janelas_teste,
                    instancia=INSTANCIA,
                    aparelho=CARGA,
                    tamanho_janela=TAMANHO_JANELA,
                    taxa=TAXA,
                    eliminar_janelas_vazias=True
                )
                
                print("     -> Detalhes da amostragem (lotes):")
                print("     ---")
                for item in Counter(y_teste).items():
                    print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_teste)*100,1)}%)" )
                print()

                # Treinando modelo
                dlafe = DLAFE(
                    feature_extractor=modelo_extrator,
                    preprocess_input=preprocessamento_extrator,
                    classifier=clone(modelo),
                    rp_params = PARAMETROS_RP,
                    input_shape = TAMANHO_IMAGEM_DLAFE,
                    normalize=False
                )
                dlafe.fit(X, y)

                # Prevendo conjunto de teste
                y_hat = dlafe.predict(X_teste)

                # Incrementando resultados
                resultados_modelo["appliance"].append(rotulo_aparelho)
                resultados_modelo["fold"].append(it+1)
                resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
                resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
                resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
                resultados_modelo["base"].append("teste")
                resultados_modelo["arquitetura"].append(arquitetura.__name__) # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                
                reset_tf_session(model_name='dlafe')
                
                print()
                print("   - Final Results:")
                print("   ---")
                print()
                
                print("***** TRAIN *****")
                print("      -> Classification Report:")
                print()
                print(classification_report(y_true, y_pred))
                print("      -> Confusion Matrix:")
                print()
                print(confusion_matrix(y_true, y_pred))
                print()
                
                print("***** TEST *****")
                print("      -> Classification Report:")
                print()
                print(classification_report(y_teste, y_hat))
                print("      -> Confusion Matrix:")
                print()
                print(confusion_matrix(y_teste, y_hat))
                print()

        except Exception as e:
            print(f"Não foi possível carregar o modelo {arquitetura.__name__}: {str(e)}")
    
# Consolidating DataFrame
df_resultados = pd.DataFrame(resultados_modelo)
df_resultados.to_excel(os.path.join(caminho_dados_notebook, "df_resultados_xgboost.xlsx"))
    
print("############################## FINAL MODEL RESULTS ##############################")
display(df_resultados.groupby(["appliance","base", "arquitetura"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))

# Avaliando modelo extrator `ResNet50`...

* Aparelho `dish_washer - 9`...

   - Base de TREINO

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
        - Classe `0`: 217 amostras (94.8%)
        - Classe `1`: 12 amostras (5.2%)

     -> Avaliando modelo (CV - 10 folds)...



  0%|          | 0/10 [00:00<?, ?it/s]

[03:51:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[03:52:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[03:52:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

  0%|          | 0/10 [00:00<?, ?it/s]

[03:53:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[03:53:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[03:53:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

...

  0%|          | 0/10 [00:00<?, ?it/s]

[09:01:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[09:02:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[09:03:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

  0%|          | 0/10 [00:00<?, ?it/s]

[09:14:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[09:15:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Virtual devices cannot be modified after being initialized
[09:15:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_me

acc                      \
                                             mean       std       max   
appliance         base   arquitetura                                    
dish_washer - 9   teste  DenseNet121     0.932432       NaN  0.932432   
                         DenseNet169     0.932432       NaN  0.932432   
                         DenseNet201     0.932432       NaN  0.932432   
                         EfficientNetB0  0.925676       NaN  0.925676   
                         EfficientNetB1  0.932432       NaN  0.932432   
...                                           ...       ...       ...   
washer_dryer - 14 treino ResNet50        0.982609  0.022452  1.000000   
                         ResNet50V2      0.978261  0.022915  1.000000   
                         VGG16           0.978261  0.030744  1.000000   
                         VGG19           0.973913  0.030400  1.000000   
                         Xception        0.978261  0.022915  1.000000   

                                                         f1            \
                                              min      mean       std   
appliance         base   arquitetura                                    
dish_washer - 9   teste  DenseNet121     0.932432  0.482517       NaN   
                         DenseNet169     0.932432  0.482517       NaN   
                         DenseNet201     0.932432  0.482517       NaN   
                         EfficientNetB0  0.925676  0.480702       NaN   
                         EfficientNetB1  0.932432  0.482517       NaN   
...                                           ...       ...       ...   
washer_dryer - 14 treino ResNet50        0.956522  0.895401  0.165728   
                         ResNet50V2      0.956522  0.877571  0.162789   
                         VGG16           0.913043  0.860957  0.212061   
                         VGG19           0.913043  0.843128  0.206494   
                         Xception        0.956522  0.844289  0.204227   

                                                                  auc  \
                                              max       min      mean   
appliance         base   arquitetura                                    
dish_washer - 9   teste  DenseNet121     0.482517  0.482517  0.500000   
                         DenseNet169     0.482517  0.482517  0.500000   
                         DenseNet201     0.482517  0.482517  0.500000   
                         EfficientNetB0  0.480702  0.480702  0.496377   
                         EfficientNetB1  0.482517  0.482517  0.500000   
...                                           ...       ...       ...   
washer_dryer - 14 treino ResNet50        1.000000  0.488889  0.875000   
                         ResNet50V2      1.000000  0.488889  0.895455   
                         VGG16           1.000000  0.477273  0.850000   
                         VGG19           1.000000  0.477273  0.822727   
                         Xception        1.000000  0.488889  0.847727   

                                                                       
                                              std       max       min  
appliance         base   arquitetura                                   
dish_washer - 9   teste  DenseNet121          NaN  0.500000  0.500000  
                         DenseNet169          NaN  0.500000  0.500000  
                         DenseNet201          NaN  0.500000  0.500000  
                         EfficientNetB0       NaN  0.496377  0.496377  
                         EfficientNetB1       NaN  0.500000  0.500000  
...                                           ...       ...       ...  
washer_dryer - 14 treino ResNet50        0.176777  1.000000  0.500000  
                         ResNet50V2      0.172155  1.000000  0.500000  
                         VGG16           0.210819  1.000000  0.500000  
                         VGG19           0.209891  1.000000  0.477273  
                         Xception        

## MLP

In [16]:
def class_weight(y, debug=False):
    
    # Classes distribution
    neg, pos = np.bincount(y)
    total = neg + pos

    # Scaling by total/2 helps keep the loss to a similar magnitude.
    # The sum of the weights of all examples stays the same.
    w_0 = (1 / neg)*(total)/2.0 
    w_1 = (1 / pos)*(total)/2.0

    class_weight = {0: w_0, 1: w_1}

    print('Weight for class 0: {:.2f}'.format(w_0))
    print('Weight for class 1: {:.2f}'.format(w_1))
    
    return class_weight

In [17]:
def mlp(
    input_shape=TAMANHO_IMAGEM_DLAFE,
    metrics=[
        tf.keras.metrics.TruePositives(name='tp'),
        tf.keras.metrics.FalsePositives(name='fp'),
        tf.keras.metrics.TrueNegatives(name='tn'),
        tf.keras.metrics.FalseNegatives(name='fn'), 
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ], 
    output_bias=None
):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    model = keras.Sequential([
      keras.layers.Dense(10, activation='relu', input_shape=input_shape),
      # keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias),
    ])

    model.compile(
      optimizer=keras.optimizers.Adam(lr=0.001),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics
    )

    return model

In [18]:
modelo = MLPClassifier(alpha=1e-3, hidden_layer_sizes=(10,), random_state=SEED)

resultados_modelo = {
    "appliance": [], "fold": [],
    "acc": [], "f1": [], "auc": [], 
    "base": [],
    "arquitetura": [] # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
}

for a in arquiteturas_deeplearning.keys():
    # Preprocessamento da arquitetura
    preprocessamento_extrator = arquiteturas_deeplearning[a][0].preprocess_input
    # Iterando multiplos modelos da arquitetura
    for arquitetura in arquiteturas_deeplearning[a][1]:
        print(f'# Avaliando modelo extrator `{arquitetura.__name__}`...\n')
        try:
            modelo_extrator = arquitetura(
                weights='imagenet', 
                include_top=False,
                pooling='avg'
            )

            for rotulo_aparelho in aparelhos:
                
                print(f"* Aparelho `{rotulo_aparelho}`...\n")
                
                # Informacoes da carga selecionada
                CARGA = rotulo_aparelho.split(" - ")[0]
                INSTANCIA = int(rotulo_aparelho.split(" - ")[1])
                config_aparelho = df_melhores_taxas_janelas[
                    df_melhores_taxas_janelas["carga"]==rotulo_aparelho
                ].to_dict("records")[0]
                TAMANHO_JANELA = config_aparelho["janela"]

                #######################################################################
                #                AVALIACAO 1 - Base de treino / CV                    #
                #######################################################################
                # Extrair series divididas em janelas para cada medidor
                print("   - Base de TREINO\n")
                print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
                    TAXA, TAMANHO_JANELA
                ))
                X, y = carregar_dados_aparelho(
                    janelas=janelas_treino,
                    instancia=INSTANCIA,
                    aparelho=CARGA,
                    tamanho_janela=TAMANHO_JANELA,
                    taxa=TAXA,
                    eliminar_janelas_vazias=True
                )
                print("     -> Detalhes da amostragem (lotes):")
                print("     ---")
                for item in Counter(y).items():
                    print(f"        - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y)*100,1)}%)" )
                print()
                
                y_true, y_pred  = [], []

                print(f"     -> Avaliando modelo (CV - {skf.n_splits} folds)...\n")
                for it, (idx_treino, idx_teste) in tqdm_notebook(enumerate(skf.split(X, y)), total=skf.n_splits):

                    # Preparando lotes
                    X_treino, X_teste = X[idx_treino], X[idx_teste]
                    y_treino, y_teste = y[idx_treino], y[idx_teste]

                    # Treinando modelo
                    dlafe = DLAFE(
                        feature_extractor=modelo_extrator,
                        preprocess_input=preprocessamento_extrator,
                        classifier=clone(modelo),
                        rp_params = PARAMETROS_RP,
                        input_shape = TAMANHO_IMAGEM_DLAFE,
                        normalize=False
                    )
                    dlafe.fit(X_treino, y_treino)

                    # Prevendo conjunto de teste
                    y_hat = dlafe.predict(X_teste)

                    # Incrementando resultados
                    resultados_modelo["appliance"].append(rotulo_aparelho)
                    resultados_modelo["fold"].append(it+1)
                    resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
                    resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
                    resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
                    resultados_modelo["base"].append("treino")
                    resultados_modelo["arquitetura"].append(arquitetura.__name__) # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                    
                    reset_tf_session(model_name='dlafe')

                    # Extendendo rotulos (analise global)
                    y_true.extend(y_teste)
                    y_pred.extend(y_hat)
                    
                #######################################################################
                #                 AVALIACAO 2 - Base de teste / CV                    #
                #######################################################################
                print("   - Base de TESTE\n")
                print("     -> Carregando dados (taxa={:.0f}, janela={:.0f})...".format(
                    TAXA, TAMANHO_JANELA
                ))

                # Avaliar na base de teste
                X_teste, y_teste = carregar_dados_aparelho(
                    janelas=janelas_teste,
                    instancia=INSTANCIA,
                    aparelho=CARGA,
                    tamanho_janela=TAMANHO_JANELA,
                    taxa=TAXA,
                    eliminar_janelas_vazias=True
                )
                
                print("     -> Detalhes da amostragem (lotes):")
                print("     ---")
                for item in Counter(y_teste).items():
                    print(f"       - Classe `{item[0]}`: {item[1]} amostras ({round(item[1]/len(y_teste)*100,1)}%)" )
                print()

                # Treinando modelo
                dlafe = DLAFE(
                    feature_extractor=modelo_extrator,
                    preprocess_input=preprocessamento_extrator,
                    classifier=clone(modelo),
                    rp_params = PARAMETROS_RP,
                    input_shape = TAMANHO_IMAGEM_DLAFE,
                    normalize=False
                )
                dlafe.fit(X, y)

                # Prevendo conjunto de teste
                y_hat = dlafe.predict(X_teste)

                # Incrementando resultados
                resultados_modelo["appliance"].append(rotulo_aparelho)
                resultados_modelo["fold"].append(it+1)
                resultados_modelo["acc"].append( accuracy_score(y_teste, y_hat) )
                resultados_modelo["f1"].append( f1_score(y_teste, y_hat, average="macro") )
                resultados_modelo["auc"].append(roc_auc_score(y_teste, y_hat) if np.unique(y_teste).shape[0]>1 else 0.5)
                resultados_modelo["base"].append("teste")
                resultados_modelo["arquitetura"].append(arquitetura.__name__) # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                
                reset_tf_session(model_name='dlafe')
                
                print()
                print("   - Final Results:")
                print("   ---")
                print()
                
                print("***** TRAIN *****")
                print("      -> Classification Report:")
                print()
                print(classification_report(y_true, y_pred))
                print("      -> Confusion Matrix:")
                print()
                print(confusion_matrix(y_true, y_pred))
                print()
                
                print("***** TEST *****")
                print("      -> Classification Report:")
                print()
                print(classification_report(y_teste, y_hat))
                print("      -> Confusion Matrix:")
                print()
                print(confusion_matrix(y_teste, y_hat))
                print()

        except Exception as e:
            print(f"Não foi possível carregar o modelo {arquitetura.__name__}: {str(e)}")
    
# Consolidating DataFrame
df_resultados = pd.DataFrame(resultados_modelo)
df_resultados.to_excel(os.path.join(caminho_dados_notebook, "df_resultados_mlp.xlsx"))
    
print("############################## FINAL MODEL RESULTS ##############################")
display(df_resultados.groupby(["appliance","base", "arquitetura"]).agg({
    "acc": ["mean", "std", "max", "min"],
    "f1": ["mean", "std", "max", "min"],
    "auc": ["mean", "std", "max", "min"]
}))

# Avaliando modelo extrator `ResNet50`...

* Aparelho `dish_washer - 9`...

   - Base de TREINO

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
        - Classe `0`: 217 amostras (94.8%)
        - Classe `1`: 12 amostras (5.2%)

     -> Avaliando modelo (CV - 10 folds)...



  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 138 amostras (93.2%)
       - Classe `1`: 10 amostras (6.8%)

Virtual devices cannot be modified after being initialized

   - Final 

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=720)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `1`: 392 amostras (95.6%)
       - Classe `0`: 18 amostras (4.4%)

Virtual devices cannot be modified after being initialized

   - Final R

...

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=90)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 3102 amostras (95.9%)
       - Classe `1`: 132 amostras (4.1%)

Virtual devices cannot be modified after being initialized

   - Final 

  0%|          | 0/10 [00:00<?, ?it/s]

Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
Virtual devices cannot be modified after being initialized
   - Base de TESTE

     -> Carregando dados (taxa=2, janela=2040)...
Meter 13 is in a nested meter group. Retrieving just the ElecMeter.
Meter 14 is in a nested meter group. Retrieving just the ElecMeter.

     -> Detalhes da amostragem (lotes):
     ---
       - Classe `0`: 134 amostras (90.5%)
       - Classe `1`: 14 amostras (9.5%)

Virtual devices cannot be modified after being initialized

   - Final 

acc                      \
                                             mean       std       max   
appliance         base   arquitetura                                    
dish_washer - 9   teste  DenseNet121     0.932432       NaN  0.932432   
                         DenseNet169     0.932432       NaN  0.932432   
                         DenseNet201     0.925676       NaN  0.925676   
                         EfficientNetB0  0.918919       NaN  0.918919   
                         EfficientNetB1  0.925676       NaN  0.925676   
...                                           ...       ...       ...   
washer_dryer - 14 treino ResNet50        0.977866  0.031686  1.000000   
                         ResNet50V2      0.973913  0.030400  1.000000   
                         VGG16           0.933992  0.056945  1.000000   
                         VGG19           0.947431  0.045260  1.000000   
                         Xception        0.982609  0.022452  1.000000   

                                                         f1            \
                                              min      mean       std   
appliance         base   arquitetura                                    
dish_washer - 9   teste  DenseNet121     0.932432  0.565728       NaN   
                         DenseNet169     0.932432  0.482517       NaN   
                         DenseNet201     0.925676  0.480702       NaN   
                         EfficientNetB0  0.918919  0.478873       NaN   
                         EfficientNetB1  0.925676  0.480702       NaN   
...                                           ...       ...       ...   
washer_dryer - 14 treino ResNet50        0.909091  0.885730  0.173161   
                         ResNet50V2      0.913043  0.874630  0.168389   
                         VGG16           0.818182  0.738760  0.209094   
                         VGG19           0.869565  0.747537  0.216876   
                         Xception        0.956522  0.895401  0.165728   

                                                                  auc  \
                                              max       min      mean   
appliance         base   arquitetura                                    
dish_washer - 9   teste  DenseNet121     0.565728  0.565728  0.546377   
                         DenseNet169     0.482517  0.482517  0.500000   
                         DenseNet201     0.480702  0.480702  0.496377   
                         EfficientNetB0  0.478873  0.478873  0.492754   
                         EfficientNetB1  0.480702  0.480702  0.496377   
...                                           ...       ...       ...   
washer_dryer - 14 treino ResNet50        1.000000  0.488889  0.895238   
                         ResNet50V2      1.000000  0.488889  0.892965   
                         VGG16           1.000000  0.450000  0.779004   
                         VGG19           1.000000  0.476190  0.786039   
                         Xception        1.000000  0.488889  0.875000   

                                                                       
                                              std       max       min  
appliance         base   arquitetura                                   
dish_washer - 9   teste  DenseNet121          NaN  0.546377  0.546377  
                         DenseNet169          NaN  0.500000  0.500000  
                         DenseNet201          NaN  0.496377  0.496377  
                         EfficientNetB0       NaN  0.492754  0.492754  
                         EfficientNetB1       NaN  0.496377  0.496377  
...                                           ...       ...       ...  
washer_dryer - 14 treino ResNet50        0.172407  1.000000  0.500000  
                         ResNet50V2      0.174480  1.000000  0.500000  
                         VGG16           0.233860  1.000000  0.428571  
                         VGG19           0.228581  1.000000  0.476190  
                         Xception        

# Análise dos Resultados (1)

In [20]:
df_resultados_svm = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_svm.xlsx"), engine="openpyxl")
df_resultados_svm["model"] = "SVM"

df_resultados_xgboost = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_xgboost.xlsx"), engine="openpyxl")
df_resultados_xgboost["model"] = "XGBOOST"

df_resultados_mlp = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_mlp.xlsx"), engine="openpyxl")
df_resultados_mlp["model"] = "MLP"

# df_resultados_elm = pd.read_excel(os.path.join(caminho_dados_notebook, "df_resultados_elm.xlsx"), engine="openpyxl)
# df_resultados_elm["model"] = "ELM"

df_analise = pd.concat([
    df_resultados_svm,
    df_resultados_xgboost,
    df_resultados_mlp, 
#     df_resultados_elm,  
])

print("* Análise por modelo:")
df_analise_modelo = df_analise.groupby(["model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
}).reset_index().sort_values(('f1','mean'), ascending=False).set_index("model")
display(df_analise_modelo)
df_analise_modelo.to_excel(os.path.join(caminho_dados_notebook, "df_analise_modelo.xlsx"))

print()
print("* Análise por aparelho/modelo:")
df_analise_aparelho = df_analise.groupby(["appliance","model","base"]).agg({
    "acc": ["mean","std","max","min"],
    "f1": ["mean","std","max","min"],
    "auc": ["mean","std","max","min"]
})#.reset_index().sort_values(('f1','mean'), ascending=False).set_index(["aparelho","metodologia"])
display(df_analise_aparelho)
df_analise_aparelho.to_excel(os.path.join(caminho_dados_notebook, "df_analise_aparelho.xlsx"))

* Análise por modelo:


base       acc                                      f1            \
                     mean       std       max       min      mean       std   
model                                                                         
XGBOOST   teste  0.963024  0.024670  0.998454  0.898649  0.771507  0.176793   
MLP       teste  0.959575  0.025857  0.998145  0.898649  0.763739  0.171211   
MLP      treino  0.959973  0.033911  1.000000  0.739130  0.741427  0.208550   
XGBOOST  treino  0.962871  0.030009  1.000000  0.826087  0.739922  0.214402   
SVM       teste  0.960554  0.024125  0.998145  0.912162  0.705758  0.210407   
SVM      treino  0.960900  0.028663  1.000000  0.901961  0.688566  0.224774   

                                  auc                                
              max       min      mean       std       max       min  
model                                                                
XGBOOST  0.989945  0.473310  0.748682  0.169451  0.981061  0.496269  
MLP      0.988066  0.475177  0.745792  0.164960  0.984526  0.485507  
MLP      1.000000  0.425000  0.738542  0.204560  1.000000  0.404762  
XGBOOST  1.000000  0.452381  0.732830  0.207120  1.000000  0.452381  
SVM      0.987889  0.482517  0.693147  0.195591  0.980738  0.500000  
SVM      1.000000  0.474227  0.687690  0.214405  1.000000  0.469388


* Análise por aparelho/modelo:


acc                                \
                                      mean       std       max       min   
appliance         model   base                                             
dish_washer - 9   MLP     teste   0.928907  0.010755  0.939189  0.898649   
                          treino  0.938099  0.032123  1.000000  0.739130   
                  SVM     teste   0.932432  0.000000  0.932432  0.932432   
                          treino  0.947628  0.017340  0.956522  0.913043   
                  XGBOOST teste   0.933314  0.003703  0.939189  0.925676   
                          treino  0.944183  0.021625  1.000000  0.863636   
fridge - 7        MLP     teste   0.975080  0.005145  0.982927  0.968293   
                          treino  0.961638  0.023480  1.000000  0.890625   
                  SVM     teste   0.963521  0.010015  0.980488  0.956098   
                          treino  0.937831  0.021833  1.000000  0.906250   
                  XGBOOST teste   0.975398  0.004983  0.985366  0.963415   
                          treino  0.963618  0.022760  1.000000  0.890625   
microwave - 16    MLP     teste   0.944459  0.009595  0.960366  0.923780   
                          treino  0.942152  0.023234  1.000000  0.882353   
                  SVM     teste   0.945652  0.008557  0.972561  0.939024   
                          treino  0.948613  0.013041  0.980392  0.901961   
                  XGBOOST teste   0.949099  0.009212  0.966463  0.932927   
                          treino  0.943329  0.020015  1.000000  0.882353   
washer_dryer - 13 MLP     teste   0.991732  0.009925  0.998145  0.958565   
                          treino  0.994793  0.006743  1.000000  0.960317   
                  SVM     teste   0.995537  0.003418  0.998145  0.986395   
                          treino  0.996581  0.003220  1.000000  0.986083   
                  XGBOOST teste   0.996088  0.002864  0.998454  0.988250   
                          treino  0.996970  0.002700  1.000000  0.988072   
washer_dryer - 14 MLP     teste   0.957697  0.023951  0.986486  0.912162   
                          treino  0.963181  0.039636  1.000000  0.818182   
                  SVM     teste   0.965629  0.021358  0.986486  0.912162   
                          treino  0.973844  0.029231  1.000000  0.913043   
                  XGBOOST teste   0.961222  0.024295  0.986486  0.898649   
                          treino  0.966257  0.034797  1.000000  0.826087   

                                        f1                                \
                                      mean       std       max       min   
appliance         model   base                                             
dish_washer - 9   MLP     teste   0.518127  0.044334  0.575120  0.475177   
                          treino  0.495829  0.071924  1.000000  0.425000   
                  SVM     teste   0.482517  0.000000  0.482517  0.482517   
                          treino  0.486514  0.004633  0.488889  0.477273   
                  XGBOOST teste   0.502490  0.039143  0.575120  0.480702   
                          treino  0.487755  0.034404  1.000000  0.463415   
fridge - 7        MLP     teste   0.837618  0.039309  0.882661  0.731824   
                          treino  0.834978  0.115948  1.000000  0.479675   
                  SVM     teste   0.615896  0.168673  0.865696  0.488778   
                          treino  0.599706  0.173678  1.000000  0.479675   
                  XGBOOST teste   0.834560  0.035522  0.907948  0.763197   
                          treino  0.842649  0.099621  1.000000  0.479675   
microwave - 16    MLP     teste   0.678106  0.105798  0.830894  0.484277   
                          treino  0.620437  0.149969  1.000000  0.468750   
                  SVM     teste   0.589052  0.109368  0.856413  0.484277   
                          treino  0.549896  0.121184  0.894845  0.474227   
                  XGBOOST teste   0.690043  0.091616  0.824505  0.484277   
          

# Conclusões

...

# Fim.

In [18]:
%load_ext watermark

In [19]:
%watermark -a "Diego Luiz Cavalca" -u -n -t -z -v -m -g

Author: Diego Luiz Cavalca

Last updated: Sun May 02 2021 13:36:14Hora oficial do Brasil

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.21.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
CPU cores   : 8
Architecture: 64bit

Git hash: a29eb3e98689f89f3597358428a2cab6bb3ab9b0

